In [1]:
!pip install CFGraph --upgrade -q

In [2]:
from rdflib import URIRef, Graph
from CFGraph import CFGraph
from pprint import pprint
ppl = lambda t: pprint([(str(s), str(p), str(o)) for s, p, o in list(t)], width=160)
ppt = lambda t: pprint([(str(e[0]), str(e[1])) for e in t], width=160)
pps = lambda objs: pprint(set([str(o) for o in objs]), width=160)

s1 = URIRef("http://example.org/s1")
s2 = URIRef("http://example.org/s2")
s3 = URIRef("http://example.org/s3")
s4 = URIRef("http://example.org/s4")
s5 = URIRef("http://example.org/s5")

p1 = URIRef("http://a.example/p1")
p2 = URIRef("http://a.example/p2")
p3 = URIRef("http://a.example/p3")
p4 = URIRef("http://a.example/p4")
p5 = URIRef("http://a.example/p5")

o1 = URIRef("http://example.org/o1")
o2 = URIRef("http://example.org/o2")
o3 = URIRef("http://example.org/o3")
o4 = URIRef("http://example.org/o4")
o5 = URIRef("http://example.org/o5")




**Our graph has for subjects:**

* `<"http://example.org/s1">`: an RDF collection with 8 elements - seven literals with "b" appearing twice and a URI.
* `<"http://example.org/s2">`: an RDF list with 7 elements -- six literals ("b" is a duplicate) and a URI.  This list is unordered
* `<"http://example.org/s3">`: an empty RDF collection.  This returns `rdf:nil` as a normal graph and an empty list in CGGraph
* `<"http://example.org/s4">`: an RDF collection with a single URI element
* `<"http://example.org/s5">`: a combination of an RDF collection *and* a URI.

In [3]:
rdf = f"""
<{s1}> <{p1}> ("a" "b" "c" "d" "e" <{o1}> "b").
<{s2}> <{p2}> "a", "b", "c", "d", "e", <{o2}>, "b".
<{s3}> <{p3}> ().
<{s4}> <{p4}> (<{o4}>).
<{s5}> <{p5}> (1 2), <{o5}>.
"""

print(rdf)


<http://example.org/s1> <http://a.example/p1> ("a" "b" "c" "d" "e" <http://example.org/o1> "b").
<http://example.org/s2> <http://a.example/p2> "a", "b", "c", "d", "e", <http://example.org/o2>, "b".
<http://example.org/s3> <http://a.example/p3> ().
<http://example.org/s4> <http://a.example/p4> (<http://example.org/o4>).
<http://example.org/s5> <http://a.example/p5> (1 2), <http://example.org/o5>.



In [4]:
# Load a Collection Flattening Graph
cg = CFGraph()
_ = cg.parse(data=rdf, format="turtle")

# Load a "vanilla" RDF graph for comparison
g = Graph()
_ = g.parse(data=rdf, format="turtle")

**The CGGraph returns all of the elements in the collection for the first predicate as a list.**


In [5]:
ppl(cg.triples((s1, p1, None)))

[('http://example.org/s1', 'http://a.example/p1', 'a'),
 ('http://example.org/s1', 'http://a.example/p1', 'b'),
 ('http://example.org/s1', 'http://a.example/p1', 'c'),
 ('http://example.org/s1', 'http://a.example/p1', 'd'),
 ('http://example.org/s1', 'http://a.example/p1', 'e'),
 ('http://example.org/s1', 'http://a.example/p1', 'http://example.org/o1'),
 ('http://example.org/s1', 'http://a.example/p1', 'b')]


**The Graph returns the first node**

In [6]:
ppl(g.triples((s1, p1, None)))

[('http://example.org/s1', 'http://a.example/p1', 'f1063555acde443e3abcd7d2a0716c608b1')]


**The output of the CGGraph and Graph are the same with lists**

In [7]:
ppl(cg.triples((s2, p2, None)))
print("Graphs are equal = " + str(set(cg.triples((s2, p2, None))) == set(g.triples((s2, p2, None)))))

[('http://example.org/s2', 'http://a.example/p2', 'e'),
 ('http://example.org/s2', 'http://a.example/p2', 'd'),
 ('http://example.org/s2', 'http://a.example/p2', 'b'),
 ('http://example.org/s2', 'http://a.example/p2', 'c'),
 ('http://example.org/s2', 'http://a.example/p2', 'a'),
 ('http://example.org/s2', 'http://a.example/p2', 'http://example.org/o2')]
Graphs are equal = True


**The empty list returns an empty list in CGGraph and `rdf:nil` in a plain Graph**

In [8]:
ppl(cg.triples((s3, p3, None)))
ppl(g.triples((s3, p3, None)))

[]
[('http://example.org/s3', 'http://a.example/p3', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#nil')]


**All the other *forward* graph functions behave as expected**

In [9]:
print("predicate_objects(s1)")
ppt(cg.predicate_objects(s1))
print("\nobjects(s1, p1)")
pps(cg.objects(s1, p1))

predicate_objects(s1)
[('http://a.example/p1', 'a'),
 ('http://a.example/p1', 'b'),
 ('http://a.example/p1', 'c'),
 ('http://a.example/p1', 'd'),
 ('http://a.example/p1', 'e'),
 ('http://a.example/p1', 'http://example.org/o1'),
 ('http://a.example/p1', 'b')]

objects(s1, p1)
{'e', 'http://example.org/o1', 'd', 'c', 'a', 'b'}


** Other functions behave as expected **

In [14]:
print("subject_predicates(o1)")
ppt(cg.subject_predicates(o1))
print("\nsubjects(p1, o1)")
pps(cg.subjects(p1, o1))
print("\nsubject_objects(p1)")
ppt(cg.subject_objects(p1))

subject_predicates(o1)
[('http://example.org/s1', 'http://a.example/p1')]

subjects(p1, o1)
{'http://example.org/s1'}

subject_objects(p1)
[('http://example.org/s1', 'a'),
 ('http://example.org/s1', 'b'),
 ('http://example.org/s1', 'c'),
 ('http://example.org/s1', 'd'),
 ('http://example.org/s1', 'e'),
 ('http://example.org/s1', 'http://example.org/o1'),
 ('http://example.org/s1', 'b')]
